In [1]:
import os

In [2]:
import urllib.parse as urlparse

In [3]:
import json

In [4]:
%config IPCompleter.greedy=True

In [5]:
CLIENT_ID = '9hSyjyWNQe-fdjIcuWZyFw'
CLIENT_SECRET = '310553664226487615423957757344381084034'
REDIRECT_URI = 'https://www.surveymonkey.com'
AUTH_CODE = ''

In [6]:
SM_API_BASE = "https://api.surveymonkey.com"
AUTH_CODE_ENDPOINT = "/oauth/authorize"

def oauth_dialog(client_id, redirect_uri):
    url_params = urlparse.urlencode({
        "redirect_uri": redirect_uri,
        "client_id": client_id,
        "response_type": "code"
    })

    auth_dialog_uri = SM_API_BASE + AUTH_CODE_ENDPOINT + "?" + url_params
    print("\nThe OAuth dialog url was " + auth_dialog_uri + "\n")
    return auth_dialog_uri

    # Insert code here that redirects user to OAuth Dialog url

In [7]:
dialog_url = oauth_dialog(CLIENT_ID, REDIRECT_URI)
print(dialog_url)


The OAuth dialog url was https://api.surveymonkey.com/oauth/authorize?redirect_uri=https%3A%2F%2Fwww.surveymonkey.com&client_id=9hSyjyWNQe-fdjIcuWZyFw&response_type=code

https://api.surveymonkey.com/oauth/authorize?redirect_uri=https%3A%2F%2Fwww.surveymonkey.com&client_id=9hSyjyWNQe-fdjIcuWZyFw&response_type=code


In [9]:
def handle_redirect(redirect_uri):
    # Parse authorization code out of url
    query_string = urlparse.urlsplit(redirect_uri).query
    authorization_code = urlparse.parse_qs(query_string).get("code", [])

    # parse_qs returns a list for every query param, just get the first one
    if not authorization_code:
        return

    return authorization_code[0]

In [10]:
AUTH_CODE = handle_redirect('https://www.surveymonkey.com/team/?code=eH0-Q6n1oTNLuUrmt4XCYU7HWS4J9Mc8.2uhOfuXCGfECYZRGANPBsrTJP-g-DU41WYzsjusEkflfK288HDzjUfZAxCgMBOghSVh8rnKBRETUOkJE7XVYVefGLRyl6dslpieoXwLVmKFSoo4FMLeS2wxsIENl0hhwxBcWxhlhb9TN6UZP2fwBCTKefFHmH-.e-6RtnbwmlWejMXveMrDHY46J6wuEFjsN8aGZU.eXJ84Sj8CaFsHTHSFdiSLCKHv')
print(AUTH_CODE)

eH0-Q6n1oTNLuUrmt4XCYU7HWS4J9Mc8.2uhOfuXCGfECYZRGANPBsrTJP-g-DU41WYzsjusEkflfK288HDzjUfZAxCgMBOghSVh8rnKBRETUOkJE7XVYVefGLRyl6dslpieoXwLVmKFSoo4FMLeS2wxsIENl0hhwxBcWxhlhb9TN6UZP2fwBCTKefFHmH-.e-6RtnbwmlWejMXveMrDHY46J6wuEFjsN8aGZU.eXJ84Sj8CaFsHTHSFdiSLCKHv


In [11]:
import requests

In [12]:
SM_API_BASE = "https://api.surveymonkey.com"
ACCESS_TOKEN_ENDPOINT = "/oauth/token"

def exchange_code_for_token(auth_code, client_secret, client_id, redirect_uri):
    auth_data = {
        "client_secret": client_secret,
        "code": auth_code,
        "redirect_uri": redirect_uri,
        "client_id": client_id,
        "grant_type": "authorization_code"
    }

    print(auth_data)
    access_token_uri = SM_API_BASE + ACCESS_TOKEN_ENDPOINT
    access_token_response = requests.post(access_token_uri, data=auth_data)
    access_json = access_token_response.json()
    if "access_token" in access_json:
        return access_json#["access_token"]
    else:
        print(access_json)
        return None

In [13]:
auth_token = exchange_code_for_token(AUTH_CODE, CLIENT_SECRET, CLIENT_ID, REDIRECT_URI)
print(auth_token)

{'client_secret': '310553664226487615423957757344381084034', 'code': 'eH0-Q6n1oTNLuUrmt4XCYU7HWS4J9Mc8.2uhOfuXCGfECYZRGANPBsrTJP-g-DU41WYzsjusEkflfK288HDzjUfZAxCgMBOghSVh8rnKBRETUOkJE7XVYVefGLRyl6dslpieoXwLVmKFSoo4FMLeS2wxsIENl0hhwxBcWxhlhb9TN6UZP2fwBCTKefFHmH-.e-6RtnbwmlWejMXveMrDHY46J6wuEFjsN8aGZU.eXJ84Sj8CaFsHTHSFdiSLCKHv', 'redirect_uri': 'https://www.surveymonkey.com', 'client_id': '9hSyjyWNQe-fdjIcuWZyFw', 'grant_type': 'authorization_code'}
{'access_token': 'YFQ9DTEESPqX.iysKaxlzs7DjkY-n97oxzHpRdEmQMGE8uvdaeFrIPNgIY.mdKqvTu78FGYqkHjFPh6Px01rc6cQ5v9w4BSf.1udE.4OByqIb-t.CQGxfQMgBDshlkkY', 'access_url': 'https://api.surveymonkey.com', 'token_type': 'bearer', 'expires_in': None}


In [33]:
# set default session w/token
def get_token_session():
    default_session = requests.Session()
    default_session.headers.update({
      "Authorization": "Bearer %s" % auth_token["access_token"],
      "Content-Type": "application/json"
    })
    return default_session

In [34]:
# get surveys
def get_surveys_json():
    session = get_token_session()

    url = "https://api.surveymonkey.com/v3/surveys"
    response = session.get(url)
    return response.json()

In [35]:
# get survey links
def get_survey_links():
    surveys_json = get_surveys_json()
    survey_links = surveys_json['links']
    next_link  = survey_links['next']
    print('next link: ' + next_link)
    last_link  = survey_links['last']
    print('last link: ' + last_link)
    return (next_link, last_link)

In [36]:
links = get_survey_links()
print(links)

def parse_links(page_url):
    # Parse authorization code out of url
    query_string = urlparse.urlsplit(page_url).query
    page_number = urlparse.parse_qs(query_string).get("page", [])
    return page_number

print(parse_links(links[1]))

next link: https://api.surveymonkey.net/v3/surveys?page=2&per_page=50
last link: https://api.surveymonkey.net/v3/surveys?page=2&per_page=50
('https://api.surveymonkey.net/v3/surveys?page=2&per_page=50', 'https://api.surveymonkey.net/v3/surveys?page=2&per_page=50')
['2']


In [37]:
# get survey data
def get_survey_data():
    surveys_json = get_surveys_json()
    return surveys_json['data']

In [39]:
# get survey ids
def get_survey_ids():
    survey_data = get_survey_data()
    #print(survey_data)
    # create a list of ids based on the survey data
    survey_ids = [o['id'] for o in survey_data]
    return survey_ids

In [42]:
ids = get_survey_ids()
first_id = ids[0]

# get survey details
def get_survey_details_json(survey_id):
    url = "https://api.surveymonkey.com/v3/surveys/%s/details" % (survey_id)
    session = get_token_session()
    response = session.get(url)
    return response.json()

# get first id_survey_detail
first_survey_json = get_survey_details_json(first_id)
print(first_survey_json)

{'title': 'PrIMES Mentor Application', 'nickname': '', 'language': 'en', 'folder_id': '0', 'category': '', 'question_count': 11, 'page_count': 1, 'response_count': 39, 'date_created': '2020-05-19T16:33:00', 'date_modified': '2020-06-09T13:50:00', 'id': '284469207', 'buttons_text': {'next_button': 'Next', 'prev_button': 'Prev', 'done_button': 'Done', 'exit_button': ''}, 'is_owner': True, 'footer': True, 'custom_variables': {}, 'href': 'https://api.surveymonkey.com/v3/surveys/284469207', 'analyze_url': 'https://www.surveymonkey.com/analyze/iJLMfRjXr6o36sNdngW6CtJhC8SC9_2BoS_2Fdy5xiemQvY_3D', 'edit_url': 'https://www.surveymonkey.com/create/?sm=iJLMfRjXr6o36sNdngW6CtJhC8SC9_2BoS_2Fdy5xiemQvY_3D', 'collect_url': 'https://www.surveymonkey.com/collect/list?sm=iJLMfRjXr6o36sNdngW6CtJhC8SC9_2BoS_2Fdy5xiemQvY_3D', 'summary_url': 'https://www.surveymonkey.com/summary/iJLMfRjXr6o36sNdngW6CtJhC8SC9_2BoS_2Fdy5xiemQvY_3D', 'preview': 'https://www.surveymonkey.com/r/Preview/?sm=Q2rJZL_2FX313Wz8RGVjRr

In [185]:
# get responses to first survey
def get_survey_responses_json(survey_id):
    bulk_url = "https://api.surveymonkey.com/v3/surveys/%s/responses/bulk" % (survey_id)
    bulk_response = session.get(bulk_url)
    return bulk_response.json()

print('First id: %s' % (first_id))
first_survey_data = get_survey_responses_json(first_id)['data']
first_survey_pages = [o['pages'] for o in first_survey_data]
print(first_survey_pages)
#collect_url = first_survey_json['summary_url']
#print(collect_url)

#get collect url
#collect_response = session.get(collect_url)
#print(collect_response)

First id: 284469207
[[{'id': '124625181', 'questions': [{'id': '471708122', 'answers': [{'tag_data': [], 'text': 'Radhika Sheth'}]}, {'id': '471709075', 'answers': [{'tag_data': [], 'text': 'dhika2210@gmail.com'}]}, {'id': '471708934', 'answers': [{'choice_id': '3116894909'}]}, {'id': '471718434', 'answers': [{'tag_data': [], 'text': 'Henry Ford Allegiance Health'}]}, {'id': '471731139', 'answers': [{'tag_data': [], 'text': 'Jackson, Michigan'}]}, {'id': '471722478', 'answers': [{'choice_id': '3116973822'}]}, {'id': '471724594', 'answers': [{'choice_id': '3116986828'}]}, {'id': '471728229', 'answers': [{'tag_data': [], 'text': 'Internal Medicine'}]}, {'id': '471721125', 'answers': [{'choice_id': '3116965711'}]}, {'id': '471730565', 'answers': [{'choice_id': '3117022785'}]}, {'id': '471731754', 'answers': [{'tag_data': [], 'text': "I'm an immigrant, female, PoC physician. While I have overcome a set of unique obstacles, I want to help other people like me, and learn from people who have

{'data': [{'id': '11676551065', 'recipient_id': '', 'collection_mode': 'default', 'response_status': 'completed', 'custom_value': '', 'first_name': '', 'last_name': '', 'email_address': '', 'ip_address': '216.86.72.125', 'logic_path': {}, 'metadata': {'contact': {}}, 'page_path': [], 'collector_id': '260548013', 'survey_id': '284469207', 'custom_variables': {}, 'edit_url': 'https://www.surveymonkey.com/r/?sm=_2F2XmNe8lAJDtcFP_2BkCllk5LCqzce7vnUEKXGD5_2BF1NyZC2N3pDuuJIw5QEtKD82H', 'analyze_url': 'https://www.surveymonkey.com/analyze/browse/iJLMfRjXr6o36sNdngW6CtJhC8SC9_2BoS_2Fdy5xiemQvY_3D?respondent_id=11676551065', 'total_time': 257, 'date_modified': '2020-06-08T13:44:01+00:00', 'date_created': '2020-06-08T13:39:43+00:00', 'href': 'https://api.surveymonkey.com/v3/surveys/284469207/responses/11676551065', 'pages': [{'id': '124625181', 'questions': [{'id': '471708122', 'answers': [{'tag_data': [], 'text': 'Radhika Sheth'}]}, {'id': '471709075', 'answers': [{'tag_data': [], 'text': 'dhik